In [1]:
import numpy as np
import pandas as pd
import time
from scipy import stats
from sklearn.preprocessing import MinMaxScaler
import joblib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import matplotlib
import os
# %matplotlib notebook
%matplotlib inline

In [2]:
# body_part = "head"
# file_names = ['./data/head_data_manual.log',
#               './data/head_data.log']

body_part = "shoulder_right"
file_names = ['./data/shoulder_right_elbow.log',
              './data/shoulder_right_no_elbow.log']

dataset = [pd.read_csv(f, delim_whitespace=True, header=0) for f in file_names]
dataset = pd.concat(dataset)

print('%d values'%(len(dataset)))
dataset = dataset[(np.abs(stats.zscore(dataset[["roll", "pitch", "yaw"]])) < 2).all(axis=1)]

322693 values


In [3]:
dataset = dataset.values[1:len(dataset)-1,0:]
# np.random.shuffle(dataset)

dataset = dataset[abs(dataset[:,12])!=0.0,:]
dataset = dataset[abs(dataset[:,13])!=0.0,:]
dataset = dataset[abs(dataset[:,14])!=0.0,:]
print('%d values after filtering outliers'%(len(dataset)))

euler_set = dataset[:, 12:15]
print('max euler ' + str(np.amax(euler_set)))
print('min euler ' + str(np.amin(euler_set)))
sensors_set = dataset[:, :12]

307837 values after filtering outliers
max euler 0.9078937262606296
min euler -1.8592258436868745


In [4]:
sin_cos_set = np.hstack([np.sin(euler_set), np.cos(euler_set)])

In [5]:
sensors_scaler = MinMaxScaler(feature_range=(-1., 1.))
sensors_set = sensors_scaler.fit_transform(sensors_set)

In [6]:
data_split = 0.7
split_idx = int(len(sensors_set)*data_split)

data_in_train = sensors_set[:split_idx,:]
data_out_train = sin_cos_set[:split_idx,:]
data_in_test = sensors_set[split_idx:,:]
data_out_test = sin_cos_set[split_idx:,:]

## Train model

In [7]:
from nn_model import NeuralNetworkModel

Instructions for updating:
non-resource variables are not supported in the long term


In [8]:
model_path = './output/'+body_part+'_tanh'

if not os.path.exists(model_path):
    os.makedirs(model_path)

model_name = model_path+'/best_model'
joblib.dump(sensors_scaler, model_path+'/scaler.pkl') 

model = NeuralNetworkModel(name=body_part, hidden_size=100)
model.fit(x=data_in_train, y=data_out_train, x_val=data_in_test, y_val=data_out_test, save_path=model_name, 
         iteration=3000, patience=300, batch_size=1000)

/home/roboy/anaconda3/envs/roboy/lib/python3.8/site-packages/tensorflow/python/keras/legacy_tf_layers/core.py:171: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  warnings.warn('`tf.layers.dense` is deprecated and '
/home/roboy/anaconda3/envs/roboy/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1719: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


#0000: Traing loss 0.33553, Valid loss 0.10152
#0010: Traing loss 0.00745, Valid loss 0.00840
#0020: Traing loss 0.00292, Valid loss 0.00457
#0030: Traing loss 0.00200, Valid loss 0.00330
#0040: Traing loss 0.00164, Valid loss 0.00269
#0050: Traing loss 0.00144, Valid loss 0.00231
#0060: Traing loss 0.00131, Valid loss 0.00262
#0070: Traing loss 0.00122, Valid loss 0.00194
#0080: Traing loss 0.00116, Valid loss 0.00220
#0090: Traing loss 0.00112, Valid loss 0.00229
#0100: Traing loss 0.00108, Valid loss 0.00198
#0110: Traing loss 0.00106, Valid loss 0.00226
#0120: Traing loss 0.00102, Valid loss 0.00210
#0130: Traing loss 0.00100, Valid loss 0.00225
#0140: Traing loss 0.00098, Valid loss 0.00183
#0150: Traing loss 0.00095, Valid loss 0.00183
#0160: Traing loss 0.00095, Valid loss 0.00190
#0170: Traing loss 0.00093, Valid loss 0.00175
#0180: Traing loss 0.00091, Valid loss 0.00197
#0190: Traing loss 0.00091, Valid loss 0.00195
#0200: Traing loss 0.00090, Valid loss 0.00185
#0210: Traing